# Data Drift Detection with Evidently and MLflow

This notebook demonstrates data drift detection using the Evidently library and MLflow tracking. We'll perform the following steps:

1. Set up the environment and MLflow tracking with DagsHub
2. Load the baseline data and trained model
3. Generate synthetic data with drift
4. Detect and visualize data drift
5. Detect and visualize model performance drift
6. Log drift reports to MLflow
7. Create automated drift monitoring

## Prerequisites

- Python 3.11+
- Required packages (scikit-learn, pandas, numpy, matplotlib, mlflow, dagshub, evidently)
- Completed data cleaning notebook (01_data_cleaning.ipynb)

## 1. Environment Setup

Let's start by importing the necessary libraries and setting up MLflow tracking with DagsHub.

In [ ]:
# Import necessary libraries
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import json

# For visualization
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_theme(style="whitegrid")

# Add parent directory to path to import project modules
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# For MLflow tracking
try:
    import mlflow
    import dagshub
    
    # Initialize DagsHub with MLflow tracking
    dagshub.init(
        repo_owner='yahiaehab10', 
        repo_name='MLflow_demo_MF', 
        mlflow=True
    )
    
    # Set experiment name
    mlflow.set_experiment("drift_detection")
    
    print("MLflow tracking with DagsHub initialized.")
except Exception as e:
    print(f"Warning: Could not initialize DagsHub MLflow tracking: {e}")
    print("Continuing with default MLflow tracking.")
    import mlflow
    mlflow.set_experiment("drift_detection")

# Import Evidently for drift detection
try:
    from evidently.report import Report
    from evidently.metrics import DataDriftTable, ColumnDriftMetric, DatasetDriftMetric
    from evidently.metrics import ColumnQuantileMetric, ColumnDistributionMetric
    from evidently.metrics.regression_performance import RegressionQualityMetric, RegressionPredictedVsActualPlot
    from evidently.metrics.classification_performance import ClassificationQualityMetric, ClassificationConfusionMatrix
    
    print("Evidently library imported successfully.")
except ImportError as e:
    print(f"Error: Could not import Evidently library: {e}")
    print("Please install Evidently: pip install evidently")

# Set up paths
data_dir = os.path.join('..', 'data')
raw_dir = os.path.join(data_dir, 'raw')
processed_dir = os.path.join(data_dir, 'processed')
drift_baseline_dir = os.path.join(data_dir, 'drift_baseline')
drift_results_dir = os.path.join(data_dir, 'drift_results')

# Create directories if they don't exist
os.makedirs(drift_results_dir, exist_ok=True)

print("Environment setup completed.")

## 2. Load Baseline Data and Model

Let's load the baseline data created in the previous notebook, as well as a simple trained model to test for model performance drift.

In [ ]:
# Load baseline statistics and data
try:
    baseline_stats = joblib.load(os.path.join(drift_baseline_dir, "diabetes_baseline_stats.joblib"))
    baseline_data = pd.read_csv(os.path.join(drift_baseline_dir, "diabetes_baseline_sample.csv"))
    print("Baseline data and statistics loaded successfully.")
except FileNotFoundError:
    print("Baseline data or statistics not found. Please run the data cleaning notebook first.")
    baseline_stats = None
    baseline_data = None

# Load training and testing data
try:
    X_train = pd.read_csv(os.path.join(processed_dir, "diabetes_X_train.csv"))
    X_test = pd.read_csv(os.path.join(processed_dir, "diabetes_X_test.csv"))
    y_train = pd.read_csv(os.path.join(processed_dir, "diabetes_y_train.csv")).iloc[:, 0]
    y_test = pd.read_csv(os.path.join(processed_dir, "diabetes_y_test.csv")).iloc[:, 0]
    print("Training and testing data loaded successfully.")
except FileNotFoundError:
    print("Training or testing data not found. Please run the data cleaning notebook first.")
    X_train, X_test, y_train, y_test = None, None, None, None

# Train a simple model for performance drift demonstration
from sklearn.ensemble import RandomForestRegressor

with mlflow.start_run(run_name="train_reference_model"):
    # Set model parameters
    params = {
        'n_estimators': 100,
        'max_depth': 10,
        'random_state': 42
    }
    
    # Log parameters
    for param, value in params.items():
        mlflow.log_param(param, value)
    
    # Train model
    model = RandomForestRegressor(**params)
    model.fit(X_train, y_train)
    
    # Evaluate on test data
    y_pred = model.predict(X_test)
    from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
    
    # Calculate metrics
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Log metrics
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("mae", mae)
    mlflow.log_metric("r2", r2)
    
    # Save model
    model_path = os.path.join(processed_dir, "reference_model.joblib")
    joblib.dump(model, model_path)
    mlflow.log_artifact(model_path)
    
    # Register model in Model Registry
    mlflow.sklearn.log_model(
        model, 
        artifact_path="reference_model",
        registered_model_name="diabetes_reference_model"
    )
    
    print(f"Reference model trained and saved with RMSE: {rmse:.4f}, MAE: {mae:.4f}, R²: {r2:.4f}")

## 3. Generate Synthetic Data with Drift

For demonstration purposes, we'll create synthetic data with different types of drift:

1. **Concept Drift**: The relationship between input and output changes
2. **Feature Drift**: The distribution of features changes
3. **Data Quality Drift**: Missing values, outliers, etc.

In [ ]:
# Generate three types of synthetic data with drift
np.random.seed(42)

# 1. Feature Drift: shift in the distribution of features
def generate_feature_drift(X, columns_to_drift, drift_factor=0.5):
    """Generate data with feature drift by shifting the mean of selected features"""
    X_drift = X.copy()
    for col in columns_to_drift:
        # Shift the mean by drift_factor standard deviations
        shift = drift_factor * X[col].std()
        X_drift[col] = X[col] + shift
    return X_drift

# 2. Covariate Drift: change in the distribution without changing relationships
def generate_covariate_drift(X, noise_factor=0.3):
    """Generate data with covariate drift by adding noise to all features"""
    X_drift = X.copy()
    for col in X.columns:
        # Add random noise
        noise = np.random.normal(0, noise_factor * X[col].std(), size=len(X))
        X_drift[col] = X[col] + noise
    return X_drift

# 3. Data Quality Drift: introduce missing values and outliers
def generate_quality_drift(X, missing_pct=0.1, outlier_pct=0.05, outlier_factor=3):
    """Generate data with quality issues like missing values and outliers"""
    X_drift = X.copy()
    
    # Introduce missing values
    mask = np.random.choice([True, False], size=X.shape, p=[missing_pct, 1-missing_pct])
    X_drift[mask] = np.nan
    
    # Introduce outliers
    for col in X.columns:
        # Select random indices for outliers
        outlier_idx = np.random.choice(
            X.index, 
            size=int(outlier_pct * len(X)), 
            replace=False
        )
        
        # Create outliers
        X_drift.loc[outlier_idx, col] = X.loc[outlier_idx, col] * outlier_factor
    
    return X_drift

# Generate the drift datasets
columns_to_drift = ['bmi', 'bp', 's5']  # Example features to drift

X_feature_drift = generate_feature_drift(X_test, columns_to_drift, drift_factor=0.8)
X_covariate_drift = generate_covariate_drift(X_test, noise_factor=0.4)
X_quality_drift = generate_quality_drift(X_test, missing_pct=0.1, outlier_pct=0.08, outlier_factor=4)

# Save drift datasets
X_feature_drift.to_csv(os.path.join(drift_results_dir, "feature_drift_data.csv"), index=False)
X_covariate_drift.to_csv(os.path.join(drift_results_dir, "covariate_drift_data.csv"), index=False)
X_quality_drift.to_csv(os.path.join(drift_results_dir, "quality_drift_data.csv"), index=False)

# Fill missing values in quality drift dataset for visualization
X_quality_drift_filled = X_quality_drift.fillna(X_quality_drift.mean())

# Visualize the original data vs. the drift data
plt.figure(figsize=(15, 12))

# Plot histograms for a few selected features to visualize drift
features_to_plot = ['bmi', 'bp', 's5']
drift_types = ['Feature Drift', 'Covariate Drift', 'Quality Drift']
drift_data = [X_feature_drift, X_covariate_drift, X_quality_drift_filled]

for i, feature in enumerate(features_to_plot):
    for j, (drift_type, drift_dataset) in enumerate(zip(drift_types, drift_data)):
        plt.subplot(len(features_to_plot), len(drift_types), i * len(drift_types) + j + 1)
        
        # Plot original data
        sns.histplot(X_test[feature], color='blue', alpha=0.5, label='Original', kde=True)
        
        # Plot drift data
        sns.histplot(drift_dataset[feature], color='red', alpha=0.5, label='Drift', kde=True)
        
        plt.title(f'{feature} - {drift_type}')
        plt.legend()

plt.tight_layout()
drift_viz_path = os.path.join(drift_results_dir, "drift_visualization.png")
plt.savefig(drift_viz_path)
plt.show()

# Log the generated drift data to MLflow
with mlflow.start_run(run_name="generate_drift_data"):
    mlflow.log_param("feature_drift_columns", columns_to_drift)
    mlflow.log_param("feature_drift_factor", 0.8)
    mlflow.log_param("covariate_drift_noise_factor", 0.4)
    mlflow.log_param("quality_drift_missing_pct", 0.1)
    mlflow.log_param("quality_drift_outlier_pct", 0.08)
    
    # Log datasets as artifacts
    mlflow.log_artifact(os.path.join(drift_results_dir, "feature_drift_data.csv"))
    mlflow.log_artifact(os.path.join(drift_results_dir, "covariate_drift_data.csv"))
    mlflow.log_artifact(os.path.join(drift_results_dir, "quality_drift_data.csv"))
    
    # Log visualization
    mlflow.log_artifact(drift_viz_path)
    
    print("Drift data generated and logged to MLflow.")

## 4. Detect Data Drift with Evidently

Now we'll use the Evidently library to detect and visualize the data drift between our original data and the synthetic drift data.

In [ ]:
# Define a function to detect and analyze data drift
def analyze_data_drift(reference_data, current_data, drift_type):
    """
    Detect data drift between reference and current data
    
    Args:
        reference_data: Original data (baseline)
        current_data: New data to check for drift
        drift_type: String describing the type of drift
        
    Returns:
        report: Evidently report object
        results: Dictionary with drift detection results
    """
    # Create data drift report
    data_drift_report = Report(metrics=[
        DatasetDriftMetric(),
        DataDriftTable(),
    ])
    
    # Calculate drift
    data_drift_report.run(reference_data=reference_data, current_data=current_data)
    report_dict = data_drift_report.as_dict()
    
    # Extract drift results
    results = {
        'drift_detected': report_dict['metrics'][0]['result']['dataset_drift'],
        'drift_score': report_dict['metrics'][0]['result']['dataset_drift_score'],
        'number_of_drifted_columns': report_dict['metrics'][0]['result']['number_of_drifted_columns'],
        'share_of_drifted_columns': report_dict['metrics'][0]['result']['share_of_drifted_columns'],
        'column_drift_scores': {}
    }
    
    # Extract column-level drift information
    for col_result in report_dict['metrics'][1]['result']['columns'].values():
        col_name = col_result['column_name']
        results['column_drift_scores'][col_name] = {
            'drift_detected': col_result['drift_detected'],
            'drift_score': col_result['drift_score']
        }
    
    # Save report as HTML
    report_path = os.path.join(drift_results_dir, f"{drift_type}_drift_report.html")
    data_drift_report.save_html(report_path)
    
    return data_drift_report, results, report_path

# Analyze each type of drift
drift_types = {
    'feature': X_feature_drift,
    'covariate': X_covariate_drift,
    'quality': X_quality_drift.fillna(X_quality_drift.mean())  # Fill NaNs for analysis
}

# Start MLflow run for drift detection
with mlflow.start_run(run_name="data_drift_detection"):
    all_results = {}
    
    for drift_type, drift_data in drift_types.items():
        print(f"\nAnalyzing {drift_type} drift...")
        
        # Detect drift
        report, results, report_path = analyze_data_drift(X_test, drift_data, drift_type)
        all_results[drift_type] = results
        
        # Log drift metrics to MLflow
        mlflow.log_metric(f"{drift_type}_drift_detected", int(results['drift_detected']))
        mlflow.log_metric(f"{drift_type}_drift_score", results['drift_score'])
        mlflow.log_metric(f"{drift_type}_drifted_columns_count", results['number_of_drifted_columns'])
        mlflow.log_metric(f"{drift_type}_drifted_columns_share", results['share_of_drifted_columns'])
        
        # Log most drifted columns
        top_drifted = sorted(
            [(col, info['drift_score']) for col, info in results['column_drift_scores'].items() if info['drift_detected']], 
            key=lambda x: x[1], 
            reverse=True
        )[:3]
        
        for i, (col, score) in enumerate(top_drifted):
            mlflow.log_metric(f"{drift_type}_top{i+1}_drifted_column_score", score)
            mlflow.log_param(f"{drift_type}_top{i+1}_drifted_column", col)
        
        # Log report as artifact
        mlflow.log_artifact(report_path)
        
        # Print drift summary
        print(f"Drift detected: {results['drift_detected']}")
        print(f"Drift score: {results['drift_score']:.4f}")
        print(f"Number of drifted columns: {results['number_of_drifted_columns']} " +
              f"({results['share_of_drifted_columns'] * 100:.1f}%)")
        
        if results['drift_detected']:
            print("\nTop drifted columns:")
            for col, score in top_drifted:
                print(f"  - {col}: {score:.4f}")
    
    # Save all results as JSON
    all_results_path = os.path.join(drift_results_dir, "all_drift_results.json")
    with open(all_results_path, 'w') as f:
        json.dump(all_results, f, indent=2)
    mlflow.log_artifact(all_results_path)
    
    print("\nData drift analysis completed and logged to MLflow.")

## 5. Detect Model Performance Drift

Let's analyze how data drift affects the model's performance.

In [ ]:
# Define a function to analyze model performance drift
def analyze_model_performance_drift(model, reference_data, reference_targets, 
                                   current_data, current_targets, drift_type):
    """
    Detect model performance drift
    
    Args:
        model: Trained model
        reference_data: Original data (baseline)
        reference_targets: Original targets
        current_data: New data to check for drift
        current_targets: New targets
        drift_type: String describing the type of drift
        
    Returns:
        report: Evidently report object
        results: Dictionary with performance drift results
    """
    # Generate predictions
    reference_predictions = model.predict(reference_data)
    current_predictions = model.predict(current_data)
    
    # For regression
    model_report = Report(metrics=[
        RegressionQualityMetric(),
        RegressionPredictedVsActualPlot()
    ])
    
    # Prepare data
    reference_data_with_pred = reference_data.copy()
    reference_data_with_pred['target'] = reference_targets
    reference_data_with_pred['prediction'] = reference_predictions
    
    current_data_with_pred = current_data.copy()
    current_data_with_pred['target'] = current_targets
    current_data_with_pred['prediction'] = current_predictions
    
    # Calculate performance drift
    model_report.run(reference_data=reference_data_with_pred, 
                    current_data=current_data_with_pred,
                    column_mapping={'target': 'target', 'prediction': 'prediction'})
    
    # Extract results
    report_dict = model_report.as_dict()
    reference_metrics = report_dict['metrics'][0]['result']['reference']
    current_metrics = report_dict['metrics'][0]['result']['current']
    
    results = {
        'reference': {
            'rmse': reference_metrics['rmse'],
            'mae': reference_metrics['mean_abs_error'],
            'r2': reference_metrics['r2_score'],
            'mape': reference_metrics['mean_abs_perc_error']
        },
        'current': {
            'rmse': current_metrics['rmse'],
            'mae': current_metrics['mean_abs_error'],
            'r2': current_metrics['r2_score'],
            'mape': current_metrics['mean_abs_perc_error']
        }
    }
    
    # Calculate performance changes
    for metric in results['reference'].keys():
        results[f'{metric}_change'] = results['current'][metric] - results['reference'][metric]
        results[f'{metric}_change_pct'] = (
            (results['current'][metric] - results['reference'][metric]) / 
            abs(results['reference'][metric]) * 100 if results['reference'][metric] != 0 else float('inf')
        )
    
    # Save report as HTML
    report_path = os.path.join(drift_results_dir, f"{drift_type}_performance_report.html")
    model_report.save_html(report_path)
    
    return model_report, results, report_path

# Analyze model performance for each type of drift
with mlflow.start_run(run_name="model_performance_drift"):
    performance_results = {}
    
    for drift_type, drift_data in drift_types.items():
        print(f"\nAnalyzing model performance with {drift_type} drift...")
        
        # Fill missing values if needed
        if drift_type == 'quality':
            drift_data_clean = X_quality_drift.copy()
            for col in drift_data_clean.columns:
                drift_data_clean[col] = drift_data_clean[col].fillna(X_test[col].mean())
        else:
            drift_data_clean = drift_data
        
        # Detect performance drift
        report, results, report_path = analyze_model_performance_drift(
            model, X_test, y_test, drift_data_clean, y_test, drift_type
        )
        performance_results[drift_type] = results
        
        # Log performance metrics to MLflow
        mlflow.log_metric(f"{drift_type}_reference_rmse", results['reference']['rmse'])
        mlflow.log_metric(f"{drift_type}_current_rmse", results['current']['rmse'])
        mlflow.log_metric(f"{drift_type}_rmse_change_pct", results['rmse_change_pct'])
        
        mlflow.log_metric(f"{drift_type}_reference_r2", results['reference']['r2'])
        mlflow.log_metric(f"{drift_type}_current_r2", results['current']['r2'])
        mlflow.log_metric(f"{drift_type}_r2_change_pct", results['r2_change_pct'])
        
        # Log report as artifact
        mlflow.log_artifact(report_path)
        
        # Print performance drift summary
        print(f"Original RMSE: {results['reference']['rmse']:.4f}")
        print(f"Drift RMSE: {results['current']['rmse']:.4f}")
        print(f"RMSE change: {results['rmse_change']:.4f} ({results['rmse_change_pct']:.2f}%)")
        
        print(f"Original R²: {results['reference']['r2']:.4f}")
        print(f"Drift R²: {results['current']['r2']:.4f}")
        print(f"R² change: {results['r2_change']:.4f} ({results['r2_change_pct']:.2f}%)")
    
    # Save all performance results as JSON
    perf_results_path = os.path.join(drift_results_dir, "all_performance_results.json")
    with open(perf_results_path, 'w') as f:
        json.dump(performance_results, f, indent=2)
    mlflow.log_artifact(perf_results_path)
    
    # Create a summary visualization of performance changes
    drift_types_list = list(performance_results.keys())
    metrics = ['rmse', 'r2']
    
    plt.figure(figsize=(15, 8))
    
    for i, metric in enumerate(metrics):
        plt.subplot(1, 2, i+1)
        
        # Prepare data for plotting
        reference_values = [performance_results[dt]['reference'][metric] for dt in drift_types_list]
        current_values = [performance_results[dt]['current'][metric] for dt in drift_types_list]
        
        # Create bar positions
        bar_positions = np.arange(len(drift_types_list))
        width = 0.35
        
        # Create grouped bars
        plt.bar(bar_positions - width/2, reference_values, width, label='Original', alpha=0.7)
        plt.bar(bar_positions + width/2, current_values, width, label='Drift', alpha=0.7)
        
        # Add labels and title
        plt.xlabel('Drift Type')
        plt.ylabel(metric.upper())
        plt.title(f'Impact of Drift on {metric.upper()}')
        plt.xticks(bar_positions, drift_types_list)
        plt.legend()
        
        # Add value labels
        for j, (ref, curr) in enumerate(zip(reference_values, current_values)):
            plt.text(j - width/2, ref, f'{ref:.3f}', ha='center', va='bottom')
            plt.text(j + width/2, curr, f'{curr:.3f}', ha='center', va='bottom')
    
    plt.tight_layout()
    
    # Save and log the visualization
    perf_viz_path = os.path.join(drift_results_dir, "performance_drift_visualization.png")
    plt.savefig(perf_viz_path)
    mlflow.log_artifact(perf_viz_path)
    plt.show()
    
    print("\nModel performance drift analysis completed and logged to MLflow.")

## 6. Setting Up Automated Drift Monitoring

Let's create a simple function that can be used for automated drift monitoring in a production environment.

In [ ]:
# Create a monitoring function that can be used in production
def monitor_drift(reference_data, current_data, model, reference_targets=None, current_targets=None,
                 drift_threshold=0.1, performance_threshold=0.1, log_mlflow=True):
    """
    Monitor data and performance drift in production
    
    Args:
        reference_data: Baseline data
        current_data: New production data
        model: Trained model
        reference_targets: Baseline targets (for performance monitoring)
        current_targets: New targets (for performance monitoring)
        drift_threshold: Threshold for data drift alerts
        performance_threshold: Threshold for performance drift alerts
        log_mlflow: Whether to log results to MLflow
    
    Returns:
        dict: Monitoring results
    """
    results = {
        'timestamp': pd.Timestamp.now(),
        'data_drift': {},
        'performance_drift': None,
        'alerts': []
    }
    
    # Check for missing values in current data
    missing_values = current_data.isnull().sum()
    missing_percentage = (missing_values / len(current_data)) * 100
    
    if missing_percentage.max() > 5:
        results['alerts'].append(f"High percentage of missing values detected: {missing_percentage.max():.2f}%")
    
    # Create data drift report
    data_drift_report = Report(metrics=[
        DatasetDriftMetric(),
        DataDriftTable(),
    ])
    
    # Calculate drift
    data_drift_report.run(reference_data=reference_data, current_data=current_data)
    report_dict = data_drift_report.as_dict()
    
    # Extract drift results
    results['data_drift'] = {
        'drift_detected': report_dict['metrics'][0]['result']['dataset_drift'],
        'drift_score': report_dict['metrics'][0]['result']['dataset_drift_score'],
        'number_of_drifted_columns': report_dict['metrics'][0]['result']['number_of_drifted_columns'],
        'share_of_drifted_columns': report_dict['metrics'][0]['result']['share_of_drifted_columns'],
        'column_drift_scores': {}
    }
    
    # Extract column-level drift information
    for col_result in report_dict['metrics'][1]['result']['columns'].values():
        col_name = col_result['column_name']
        results['data_drift']['column_drift_scores'][col_name] = {
            'drift_detected': col_result['drift_detected'],
            'drift_score': col_result['drift_score']
        }
    
    # Add data drift alerts
    if results['data_drift']['drift_detected']:
        results['alerts'].append(
            f"Data drift detected with score {results['data_drift']['drift_score']:.4f}"
        )
        
        # Get top drifted columns
        top_drifted = sorted(
            [(col, info['drift_score']) 
             for col, info in results['data_drift']['column_drift_scores'].items() 
             if info['drift_detected']], 
            key=lambda x: x[1], 
            reverse=True
        )[:3]
        
        if top_drifted:
            drift_cols = ", ".join([f"{col} ({score:.4f})" for col, score in top_drifted])
            results['alerts'].append(f"Top drifted columns: {drift_cols}")
    
    # Check model performance drift if targets are provided
    if reference_targets is not None and current_targets is not None:
        # Make predictions
        reference_predictions = model.predict(reference_data)
        current_predictions = model.predict(current_data)
        
        # Calculate metrics
        from sklearn.metrics import mean_squared_error, r2_score
        
        reference_rmse = np.sqrt(mean_squared_error(reference_targets, reference_predictions))
        current_rmse = np.sqrt(mean_squared_error(current_targets, current_predictions))
        
        reference_r2 = r2_score(reference_targets, reference_predictions)
        current_r2 = r2_score(current_targets, current_predictions)
        
        # Calculate changes
        rmse_change = current_rmse - reference_rmse
        rmse_change_pct = (rmse_change / reference_rmse) * 100 if reference_rmse != 0 else float('inf')
        
        r2_change = current_r2 - reference_r2
        r2_change_pct = (r2_change / abs(reference_r2)) * 100 if reference_r2 != 0 else float('inf')
        
        # Store performance drift results
        results['performance_drift'] = {
            'reference_rmse': reference_rmse,
            'current_rmse': current_rmse,
            'rmse_change': rmse_change,
            'rmse_change_pct': rmse_change_pct,
            'reference_r2': reference_r2,
            'current_r2': current_r2,
            'r2_change': r2_change,
            'r2_change_pct': r2_change_pct
        }
        
        # Add performance drift alerts
        if abs(rmse_change_pct) > performance_threshold * 100:
            results['alerts'].append(
                f"Performance drift detected: RMSE changed by {rmse_change_pct:.2f}%"
            )
        
        if abs(r2_change) > performance_threshold:
            results['alerts'].append(
                f"Performance drift detected: R² changed by {r2_change:.4f}"
            )
    
    # Log to MLflow if requested
    if log_mlflow:
        timestamp_str = pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')
        
        with mlflow.start_run(run_name=f"drift_monitoring_{timestamp_str}"):
            # Log drift metrics
            mlflow.log_metric("data_drift_detected", int(results['data_drift']['drift_detected']))
            mlflow.log_metric("data_drift_score", results['data_drift']['drift_score'])
            mlflow.log_metric("drifted_columns_count", results['data_drift']['number_of_drifted_columns'])
            
            # Log performance metrics if available
            if results['performance_drift'] is not None:
                mlflow.log_metric("current_rmse", results['performance_drift']['current_rmse'])
                mlflow.log_metric("rmse_change_pct", results['performance_drift']['rmse_change_pct'])
                mlflow.log_metric("current_r2", results['performance_drift']['current_r2'])
                mlflow.log_metric("r2_change", results['performance_drift']['r2_change'])
            
            # Log alerts as parameters
            for i, alert in enumerate(results['alerts']):
                mlflow.log_param(f"alert_{i+1}", alert)
            
            # Save and log results as JSON
            monitor_results_path = os.path.join(drift_results_dir, f"monitoring_results_{timestamp_str}.json")
            with open(monitor_results_path, 'w') as f:
                # Convert timestamp to string for JSON serialization
                results_json = results.copy()
                results_json['timestamp'] = results_json['timestamp'].strftime('%Y-%m-%d %H:%M:%S')
                json.dump(results_json, f, indent=2)
            
            mlflow.log_artifact(monitor_results_path)
    
    return results

# Test the monitoring function with our synthetic drift data
print("Testing automated drift monitoring function...")

# Test with feature drift data
monitoring_results = monitor_drift(
    reference_data=X_test,
    current_data=X_feature_drift,
    model=model,
    reference_targets=y_test,
    current_targets=y_test,
    drift_threshold=0.05,
    performance_threshold=0.05,
    log_mlflow=True
)

# Print monitoring results
print("\nMonitoring Results:")
print(f"Timestamp: {monitoring_results['timestamp']}")
print(f"Data drift detected: {monitoring_results['data_drift']['drift_detected']}")
print(f"Data drift score: {monitoring_results['data_drift']['drift_score']:.4f}")

if monitoring_results['performance_drift'] is not None:
    print(f"RMSE change: {monitoring_results['performance_drift']['rmse_change_pct']:.2f}%")
    print(f"R² change: {monitoring_results['performance_drift']['r2_change']:.4f}")

print("\nAlerts:")
for alert in monitoring_results['alerts']:
    print(f"⚠️ {alert}")

# Save the monitoring function to a Python module
monitoring_module = """
\"\"\"
Drift monitoring module for production use.
\"\"\"
import pandas as pd
import numpy as np
import mlflow
from evidently.report import Report
from evidently.metrics import DataDriftTable, DatasetDriftMetric
from sklearn.metrics import mean_squared_error, r2_score
import json
import os

def monitor_drift(reference_data, current_data, model, reference_targets=None, current_targets=None,
                 drift_threshold=0.1, performance_threshold=0.1, log_mlflow=True, output_dir=None):
    \"\"\"
    Monitor data and performance drift in production
    
    Args:
        reference_data: Baseline data
        current_data: New production data
        model: Trained model
        reference_targets: Baseline targets (for performance monitoring)
        current_targets: New targets (for performance monitoring)
        drift_threshold: Threshold for data drift alerts
        performance_threshold: Threshold for performance drift alerts
        log_mlflow: Whether to log results to MLflow
        output_dir: Directory to save results
    
    Returns:
        dict: Monitoring results
    \"\"\"
    results = {
        'timestamp': pd.Timestamp.now(),
        'data_drift': {},
        'performance_drift': None,
        'alerts': []
    }
    
    # Check for missing values in current data
    missing_values = current_data.isnull().sum()
    missing_percentage = (missing_values / len(current_data)) * 100
    
    if missing_percentage.max() > 5:
        results['alerts'].append(f"High percentage of missing values detected: {missing_percentage.max():.2f}%")
    
    # Create data drift report
    data_drift_report = Report(metrics=[
        DatasetDriftMetric(),
        DataDriftTable(),
    ])
    
    # Calculate drift
    data_drift_report.run(reference_data=reference_data, current_data=current_data)
    report_dict = data_drift_report.as_dict()
    
    # Extract drift results
    results['data_drift'] = {
        'drift_detected': report_dict['metrics'][0]['result']['dataset_drift'],
        'drift_score': report_dict['metrics'][0]['result']['dataset_drift_score'],
        'number_of_drifted_columns': report_dict['metrics'][0]['result']['number_of_drifted_columns'],
        'share_of_drifted_columns': report_dict['metrics'][0]['result']['share_of_drifted_columns'],
        'column_drift_scores': {}
    }
    
    # Extract column-level drift information
    for col_result in report_dict['metrics'][1]['result']['columns'].values():
        col_name = col_result['column_name']
        results['data_drift']['column_drift_scores'][col_name] = {
            'drift_detected': col_result['drift_detected'],
            'drift_score': col_result['drift_score']
        }
    
    # Add data drift alerts
    if results['data_drift']['drift_detected']:
        results['alerts'].append(
            f"Data drift detected with score {results['data_drift']['drift_score']:.4f}"
        )
        
        # Get top drifted columns
        top_drifted = sorted(
            [(col, info['drift_score']) 
             for col, info in results['data_drift']['column_drift_scores'].items() 
             if info['drift_detected']], 
            key=lambda x: x[1], 
            reverse=True
        )[:3]
        
        if top_drifted:
            drift_cols = ", ".join([f"{col} ({score:.4f})" for col, score in top_drifted])
            results['alerts'].append(f"Top drifted columns: {drift_cols}")
    
    # Check model performance drift if targets are provided
    if reference_targets is not None and current_targets is not None:
        # Make predictions
        reference_predictions = model.predict(reference_data)
        current_predictions = model.predict(current_data)
        
        # Calculate metrics
        reference_rmse = np.sqrt(mean_squared_error(reference_targets, reference_predictions))
        current_rmse = np.sqrt(mean_squared_error(current_targets, current_predictions))
        
        reference_r2 = r2_score(reference_targets, reference_predictions)
        current_r2 = r2_score(current_targets, current_predictions)
        
        # Calculate changes
        rmse_change = current_rmse - reference_rmse
        rmse_change_pct = (rmse_change / reference_rmse) * 100 if reference_rmse != 0 else float('inf')
        
        r2_change = current_r2 - reference_r2
        r2_change_pct = (r2_change / abs(reference_r2)) * 100 if reference_r2 != 0 else float('inf')
        
        # Store performance drift results
        results['performance_drift'] = {
            'reference_rmse': reference_rmse,
            'current_rmse': current_rmse,
            'rmse_change': rmse_change,
            'rmse_change_pct': rmse_change_pct,
            'reference_r2': reference_r2,
            'current_r2': current_r2,
            'r2_change': r2_change,
            'r2_change_pct': r2_change_pct
        }
        
        # Add performance drift alerts
        if abs(rmse_change_pct) > performance_threshold * 100:
            results['alerts'].append(
                f"Performance drift detected: RMSE changed by {rmse_change_pct:.2f}%"
            )
        
        if abs(r2_change) > performance_threshold:
            results['alerts'].append(
                f"Performance drift detected: R² changed by {r2_change:.4f}"
            )
    
    # Log to MLflow if requested
    if log_mlflow:
        timestamp_str = pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')
        
        with mlflow.start_run(run_name=f"drift_monitoring_{timestamp_str}"):
            # Log drift metrics
            mlflow.log_metric("data_drift_detected", int(results['data_drift']['drift_detected']))
            mlflow.log_metric("data_drift_score", results['data_drift']['drift_score'])
            mlflow.log_metric("drifted_columns_count", results['data_drift']['number_of_drifted_columns'])
            
            # Log performance metrics if available
            if results['performance_drift'] is not None:
                mlflow.log_metric("current_rmse", results['performance_drift']['current_rmse'])
                mlflow.log_metric("rmse_change_pct", results['performance_drift']['rmse_change_pct'])
                mlflow.log_metric("current_r2", results['performance_drift']['current_r2'])
                mlflow.log_metric("r2_change", results['performance_drift']['r2_change'])
            
            # Log alerts as parameters
            for i, alert in enumerate(results['alerts']):
                mlflow.log_param(f"alert_{i+1}", alert)
            
            # Save and log results as JSON
            if output_dir is None:
                output_dir = 'drift_results'
                os.makedirs(output_dir, exist_ok=True)
                
            monitor_results_path = os.path.join(output_dir, f"monitoring_results_{timestamp_str}.json")
            with open(monitor_results_path, 'w') as f:
                # Convert timestamp to string for JSON serialization
                results_json = results.copy()
                results_json['timestamp'] = results_json['timestamp'].strftime('%Y-%m-%d %H:%M:%S')
                json.dump(results_json, f, indent=2)
            
            mlflow.log_artifact(monitor_results_path)
    
    return results
"""

# Save the monitoring module
with open(os.path.join('..', 'src', 'drift_monitoring.py'), 'w') as f:
    f.write(monitoring_module)

print("\nDrift monitoring module saved to '../src/drift_monitoring.py'")
print("This module can be imported and used for automated drift monitoring in production.")

## 7. Summary

In this notebook, we've:

1. Set up the environment and MLflow tracking with DagsHub
2. Loaded the baseline data and trained a simple model
3. Generated synthetic data with different types of drift
4. Detected and visualized data drift using Evidently
5. Analyzed the impact of drift on model performance
6. Created an automated drift monitoring solution for production use

All the drift reports and visualizations are logged to MLflow and can be viewed in the MLflow UI. The monitoring function we created can be used in a production environment to continuously monitor for drift and alert when necessary.

Next steps:
- Proceed to model training notebook (03_model_training.ipynb)
- Implement the drift monitoring in a production pipeline